In [1]:
import pandas as pd

In [2]:
from game_map import local_to_global

In [33]:
from tqdm import tqdm
import json

In [4]:
!ls

99975168_steps.csv.gz
LICENSE
Untitled.ipynb
__pycache__
characters_transparent.png
game_map.py
global_coords_64_envs_8192_steps_76_games.csv.gz
index.html
kanto_big_done1.png
map_data.json
pixi.js
test.csv.gz
visualizer.js


In [5]:
column_names = ['id', 'reset', 'x', 'y', 'map']

In [6]:
df = pd.read_csv("99975168_steps.csv.gz", names=column_names)

In [7]:
df

,id,reset,x,y,map
0,session_fb32,1,3,5,40
1,session_0059,1,3,5,40
2,session_7401,1,3,5,40
3,session_f0b8,1,3,4,40
4,session_abcf,1,3,5,40
...,...,...,...,...,...
100165563,session_b66d,77,9,5,197
100165564,session_0e2b,77,2,29,22
100165565,session_1a59,77,17,17,5
100165566,session_b59c,77,17,6,17


In [8]:
100165568 / (64*77)

20325.805194805194

In [9]:
2048*10*64#*76

1310720

In [10]:
df[1310720 - 1 : 1310720 + 1]

,id,reset,x,y,map
1310719,session_d4aa,1,25,15,12
1310720,session_a1f9,2,5,13,39


In [56]:
games = 76
envs = 64
steps = 4*2048 # 256

In [57]:
chunks = []
for i in range(76):
  chunks.append(df[1310720*i:1310720*i+envs*steps])

In [58]:
raw_xy = []
for c in tqdm(chunks):
    raw_xy += c[["x","y","map"]].apply(
        lambda row: local_to_global(row["x"], row["y"], row["map"]), 
        axis=1,
        result_type="expand"
    ).values.flatten().tolist()
res = {"envs": envs, "games": games, "steps": steps, "flat_xy": raw_xy}

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 76/76 [10:54<00:00,  8.61s/it]


In [54]:
 
# Writing to sample.json
with open(f"global_coords_{envs}_envs_{steps}_steps_{games}_games.json", "w") as outfile:
    outfile.write(json.dumps(res, indent=4))

In [46]:
!du -h global_coords_64_envs_256_steps_76_games.json

 30M	global_coords_64_envs_256_steps_76_games.json
